In [20]:
#Author: Ozan Başaran 26805
import sys
import time
import numpy as np
MAX_DEPTH=6

In [2]:
class Player:
    def __init__(self, name, symbol):
        self.name = name
        self.symbol = symbol[0]  # Take the first character of the symbol
        self.score = 0

    def update_score(self, points):
        self.score += points


In [3]:
class Intersection:
    def __init__(self, value):
        self.value = int(value)
        self.total_edges = 0
        self.claimed= False

    def increment_edges(self):
        self.total_edges += 1
    def decrement_edges(self):
      self.total_edges -= 1
    def get_points(self):
        if self.claimed==False:
          if self.total_edges==self.value:
            self.claimed=True
            return self.value
          else:
            return 0
        else:
          return 0


In [4]:
def is_game_over(grid):
    for row in grid:
        if ' ' in row:
            return False
    return True


In [5]:
def print_grid(grid, intersections):
    m = len(grid)
    line = ''
    for i in range(m+1):
      for j in range(m):
        line += str(intersections[i][j].value) + '---'
      line += str(intersections[i][j+1].value)
      print(line)
        
      if i < m:
        grid_line = '|'
        for j in range(m):
          grid_line += ' ' + grid[i][j] + ' |'
        print(grid_line)
            
      line = ''
    
    print(line)


In [6]:
def get_touched_vertices(row,col, player):
    if player.symbol == '/':
        return [(row, col+1), (row+1, col)]
    else:
        return [(row, col), (row+1, col+1)]


In [ ]:
def generate_child_branches(grid, intersections, player1):
    child_moves = []
    m = len(grid)

    for row in range(m):
        for col in range(m):
            if grid[row][col] == ' ':
                new_grid = [row[:] for row in grid]  # Create a copy of the grid
                new_intersections = [[Intersection(intersections[i][j].value) for j in range(m+1)] for i in range(m+1)]  # Create a copy of the intersections

                new_grid[row][col] = player1.symbol  # Make the move for Player 1
                touched_vertices = get_touched_vertices(row, col, player1)
                for vertex in touched_vertices:
                    intersection = new_intersections[vertex[0]][vertex[1]]
                    intersection.increment_edges()
                    points = intersection.get_points()
                    if points > 0:
                        player1.update_score(points)
                        player2.update_score(-points)

                empty_cells = []
                for i in range(m):
                    for j in range(m):
                        if new_grid[i][j] == ' ':
                            empty_cells.append((i, j))

                if len(empty_cells) == 1:
                    # Only one empty cell remaining, place the other player's symbol there
                    other_player = Player("Player 2", "/") if player1.symbol == "\\" else Player("Player 2", "\\")
                    new_grid[empty_cells[0][0]][empty_cells[0][1]] = other_player.symbol
                    touched_vertices = get_touched_vertices(empty_cells[0][0], empty_cells[0][1], other_player)
                    for vertex in touched_vertices:
                        intersection = new_intersections[vertex[0]][vertex[1]]
                        intersection.increment_edges()
                        points = intersection.get_points()
                        if points > 0:
                            player1.update_score(-points)
                            player2.update_score(points)

                child_moves.append((new_grid, new_intersections))

    return child_moves









In [7]:
def generate_moves(grid, intersections, player1, player2, turn):
    moves = []
    m = len(grid)

    if turn == 'player1':
        player = player1
        other_player = player2
    else:
        player = player2
        other_player = player1

    for row in range(m):
        for col in range(m):
            if grid[row][col] == ' ':
                new_grid = [row[:] for row in grid]  # Create a copy of the grid
                new_intersections = [[Intersection(intersections[i][j].value) for j in range(m + 1)] for i in range(m + 1)]  # Create a copy of the intersections

                new_grid[row][col] = player.symbol  # Make the move for the current player
                touched_vertices = get_touched_vertices(row, col, player)

                for vertex in touched_vertices:
                    intersection = new_intersections[vertex[0]][vertex[1]]
                    intersection.increment_edges()
                    points = intersection.get_points()
                    if points > 0:
                        player.update_score(points)
                        other_player.update_score(-points)

                empty_cells = []
                for i in range(m):
                    for j in range(m):
                        if new_grid[i][j] == ' ':
                            empty_cells.append((i, j))

                if len(empty_cells) == 1:
                    # Only one empty cell remaining, place the other player's symbol there
                    new_grid[empty_cells[0][0]][empty_cells[0][1]] = other_player.symbol
                    touched_vertices = get_touched_vertices(empty_cells[0][0], empty_cells[0][1], other_player)

                    for vertex in touched_vertices:
                        intersection = new_intersections[vertex[0]][vertex[1]]
                        intersection.increment_edges()
                        points = intersection.get_points()
                        if points > 0:
                            other_player.update_score(points)
                            player.update_score(-points)

                moves.append((new_grid, new_intersections))

                # Revert the scores to the original state after evaluating the move
                player.update_score(-points)
                other_player.update_score(points)

    return moves


In [8]:
def toggle_turn(turn):
    if turn == 'player1':
        return 'player2'
    else:
        return 'player1'


In [9]:
def minimax(depth, counter, grid, intersections, player1, player2, alpha, beta, maximizing_player, turn):
    if counter >= MAX_DEPTH or is_game_over(grid):
        if maximizing_player:
            return player2.score
        else:
            return player1.score

    if maximizing_player:
        max_eval = float('-inf')
        branches = generate_moves(grid, intersections, player1, player2, turn)

        for branch in branches:
            child_grid, child_intersections = branch
            eval = minimax(depth + 1, counter + 1, child_grid, child_intersections, player1, player2, alpha, beta, False, toggle_turn(turn))
            max_eval = max(max_eval, eval)
            alpha = max(alpha, eval)
            if beta <= alpha:
                break  # Beta cutoff
        return max_eval
    else:
        min_eval = float('inf')
        branches = generate_moves(grid, intersections, player1, player2, turn)

        for branch in branches:
            child_grid, child_intersections = branch
            eval = minimax(depth + 1, counter + 1, child_grid, child_intersections, player1, player2, alpha, beta, True, toggle_turn(turn))
            min_eval = min(min_eval, eval)
            beta = min(beta, eval)
            if beta <= alpha:
                break  # Alpha cutoff
        return min_eval


In [10]:
import copy

def get_move(grid, intersections, player1, player2, turn, depth):
    if depth == 0 or is_game_over(grid):
        return None, None, grid, intersections  # Depth limit reached or game is over, no move to make

    if turn == 'player1':
        print("Player 1's turn")
        row = int(input("Enter row: "))
        col = int(input("Enter column: "))
        if grid[row][col] == ' ':
            grid[row][col] = player1.symbol
            touched_vertices = get_touched_vertices(row, col, player1)
            for vertex in touched_vertices:
                intersection = intersections[vertex[0]][vertex[1]]
                intersection.increment_edges()
                points = intersection.get_points()
                if points > 0:
                    player1.update_score(points)
                    player2.update_score(-points)
            print("Scoreboard:")
            print("Player 1: ", player1.score)
            print("Player 2: ", player2.score)
            print_grid(grid, intersections)

            return 'player2', depth - 1, grid, intersections
        else:
            print("Invalid move. Please choose an empty cell.")
            return 'player1', depth, grid, intersections

    else:
        print("Player 2's turn")
        alpha = float('-inf')
        beta = float('inf')
        best_score = float('-inf')
        best_move = None
        counter=0
        branches = generate_moves(grid, copy.deepcopy(intersections), copy.deepcopy(player1), copy.deepcopy(player2), turn)
        for branch in branches:
            child_grid, child_intersections = branch
            score = minimax(depth - 1, counter,child_grid, child_intersections, copy.deepcopy(player1), copy.deepcopy(player2), alpha, beta, True, turn)
            counter=0
            if score > best_score:
                best_score = score
                best_move = branch

        if best_move is not None:
            child_grid, child_intersections = best_move
            row, col = get_move_coordinates(grid, child_grid)  # Get the coordinates of the best move
            grid[row][col] = player2.symbol  # Update grid with the best move
            touched_vertices = get_touched_vertices(row, col, player2)
            for vertex in touched_vertices:
                intersection = intersections[vertex[0]][vertex[1]]
                intersection.increment_edges()
                points = intersection.get_points()
                if points > 0:
                    player2.update_score(points)
                    player1.update_score(-points)
            print("Scoreboard:")
            print("Player 1: ", player1.score)
            print("Player 2: ", player2.score)
            print_grid(grid, intersections)

        return 'player1', depth - 1, grid, intersections


def get_move_coordinates(old_grid, new_grid):
    for row in range(len(old_grid)):
        for col in range(len(old_grid[row])):
            if old_grid[row][col] != new_grid[row][col]:
                return row, col
    return None, None


In [19]:
m =4 
#m=2 
grid = [[' ' for _ in range(m)] for _ in range(m)]
#vertices=np.array([[1,2,0],[1,1,1],[1,1,1]])
#vertices = np.array([[1, 2, 0],[1, 2, 1],[1, 1, 1]])
vertices=np.array([[0,2,0,1,0],[1,1,0,2,1],[1,0,2,3,1],[1,2,2,0,1],[2,0,2,1,0]])
intersections = [[Intersection(str(vertices[i, j])) for j in range(m + 1)] for i in range(m + 1)]
print_grid(grid,intersections)
player1 = Player("Player 1", '/')
player2 = Player("Player 2", '\\')
turn="player1"
depth=m*m
while is_game_over(grid) == False:
  turn,depth,grid,intersections=get_move(grid,intersections,player1,player2,turn,depth)


0---2---0---1---0
|   |   |   |   |
1---1---0---2---1
|   |   |   |   |
1---0---2---3---1
|   |   |   |   |
1---2---2---0---1
|   |   |   |   |
2---0---2---1---0

Player 1's turn
Enter row: 3
Enter column: 3
Scoreboard:
Player 1:  2
Player 2:  -2
0---2---0---1---0
|   |   |   |   |
1---1---0---2---1
|   |   |   |   |
1---0---2---3---1
|   |   |   |   |
1---2---2---0---1
|   |   |   | / |
2---0---2---1---0

Player 2's turn
Scoreboard:
Player 1:  1
Player 2:  -1
0---2---0---1---0
|   |   |   |   |
1---1---0---2---1
|   |   |   | \ |
1---0---2---3---1
|   |   |   |   |
1---2---2---0---1
|   |   |   | / |
2---0---2---1---0

Player 1's turn
Enter row: 1
Enter column: 0
Scoreboard:
Player 1:  3
Player 2:  -3
0---2---0---1---0
|   |   |   |   |
1---1---0---2---1
| / |   |   | \ |
1---0---2---3---1
|   |   |   |   |
1---2---2---0---1
|   |   |   | / |
2---0---2---1---0

Player 2's turn
Scoreboard:
Player 1:  3
Player 2:  -3
0---2---0---1---0
|   |   |   |   |
1---1---0---2---1
| / |   |   | \ 